**MACHINE LEARNING WITH PYSPARK**
+ USING DATABRICK TOOL
+ https://community.cloud.databricks.com/


#### Check if patient is **Hep** or No based on parameters

##### ML with pyspark
+ Classify predict

###### Datasource
+ **[Data Set link](https://archive.ics.uci.edu/ml/datasets/HCV+data)**

####### Abstract
+ **The data set contains laboratory values of blood donors and Hepatitis C patients and demographic values like age**


In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = spark = SparkSession.builder.appName('ML_With_Spark').getOrCreate()

2022-06-11 23:22:30 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [23]:
spark.sparkContext.setLogLevel("ERROR") #ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN

In [4]:
! /usr/libexec/java_home -V

Matching Java Virtual Machines (2):
    18.0.1 (x86_64) "Oracle Corporation" - "OpenJDK 18.0.1" /Users/amadoudiallo/Library/Java/JavaVirtualMachines/openjdk-18.0.1/Contents/Home
    1.8.0_292 (x86_64) "AdoptOpenJDK" - "AdoptOpenJDK 8" /Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home
/Users/amadoudiallo/Library/Java/JavaVirtualMachines/openjdk-18.0.1/Contents/Home


In [46]:
## read data from filestore of databricks

df = spark.read.csv(path=r"./data/hcv.csv", inferSchema=True, header=True)
#df = spark.read.csv('data/hcv.csv', header=True, inferSchema=True)

In [47]:
df.show(6)

+---+-------------+---+---+----+----+----+----+----+-----+----+-----+----+----+
|_c0|     Category|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|
+---+-------------+---+---+----+----+----+----+----+-----+----+-----+----+----+
|  1|0=Blood Donor| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|  69|
|  2|0=Blood Donor| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|
|  3|0=Blood Donor| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|
|  4|0=Blood Donor| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|
|  5|0=Blood Donor| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|
|  6|0=Blood Donor| 32|  m|41.6|43.3|18.5|19.7|12.3| 9.92|6.05|111.0|91.0|  74|
+---+-------------+---+---+----+----+----+----+----+-----+----+-----+----+----+
only showing top 6 rows



In [48]:
# columns 
print(df.columns)

['_c0', 'Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']


In [49]:
# rearrange columns by putting category as last columns (it is the target to predict)
df = df.select('Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'Category')

In [50]:
# check the rearrangement
df.show(5)

+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|     Category|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|  69|0=Blood Donor|
| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|0=Blood Donor|
| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|0=Blood Donor|
| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|0=Blood Donor|
| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|0=Blood Donor|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
only showing top 5 rows



In [51]:
# check types
# Before inferSchema = True
df.dtypes

[('Age', 'int'),
 ('Sex', 'string'),
 ('ALB', 'string'),
 ('ALP', 'string'),
 ('ALT', 'string'),
 ('AST', 'double'),
 ('BIL', 'double'),
 ('CHE', 'double'),
 ('CHOL', 'string'),
 ('CREA', 'double'),
 ('GGT', 'double'),
 ('PROT', 'string'),
 ('Category', 'string')]

In [52]:
# check for Schema
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- ALB: string (nullable = true)
 |-- ALP: string (nullable = true)
 |-- ALT: string (nullable = true)
 |-- AST: double (nullable = true)
 |-- BIL: double (nullable = true)
 |-- CHE: double (nullable = true)
 |-- CHOL: string (nullable = true)
 |-- CREA: double (nullable = true)
 |-- GGT: double (nullable = true)
 |-- PROT: string (nullable = true)
 |-- Category: string (nullable = true)



In [53]:
# description summary
print(df.describe().show())

+-------+------------------+----+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-------------+
|summary|               Age| Sex|              ALB|               ALP|               ALT|              AST|               BIL|               CHE|              CHOL|             CREA|              GGT|             PROT|     Category|
+-------+------------------+----+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-------------+
|  count|               615| 615|              615|               615|               615|              615|               615|               615|               615|              615|              615|              615|          615|
|   mean| 47.40813008130081|null|41.62019543973941| 68.2839195979899

**Check balance using value count**

In [54]:
df.groupBy('Category').count().show()

+--------------------+-----+
|            Category|count|
+--------------------+-----+
|       0=Blood Donor|  533|
|         3=Cirrhosis|   30|
|          2=Fibrosis|   21|
|0s=suspect Blood ...|    7|
|         1=Hepatitis|   24|
+--------------------+-----+



**Feature engineering**
+ Numerical values
+ Vectorization
+ Scaling

In [55]:
df.show(5)

+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|     Category|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|  69|0=Blood Donor|
| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|0=Blood Donor|
| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|0=Blood Donor|
| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|0=Blood Donor|
| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|0=Blood Donor|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
only showing top 5 rows



**For feature enginneering, we use the ml package of pyspark for vectorization, building model, and evaluation..**

In [56]:
## import ML package
import pyspark.ml

In [57]:
dir(pyspark.ml)

['Estimator',
 'Model',
 'Pipeline',
 'PipelineModel',
 'Transformer',
 'UnaryTransformer',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'base',
 'classification',
 'clustering',
 'common',
 'evaluation',
 'feature',
 'fpm',
 'image',
 'linalg',
 'param',
 'pipeline',
 'recommendation',
 'regression',
 'stat',
 'tuning',
 'util',
 'wrapper']

In [58]:
## Unique values for sex
df.select('Sex').distinct().show() # we have two distinct value for sex .. 'm' as male and 'f' as female

+---+
|Sex|
+---+
|  m|
|  f|
+---+



In [59]:
# Vectorization

from pyspark.ml.feature import VectorAssembler, StringIndexer

In [60]:
## Convert this column into Numerical value
## label encoding (0 or 1 of value Sex column)

genderEncoder = StringIndexer(inputCol='Sex', outputCol= 'Gender').fit(df) # we use Sex column to generate Gender column.


In [61]:
df = genderEncoder.transform(df)

In [62]:
## Category column
## label encoding
catEncoder = StringIndexer(inputCol='Category', outputCol='Target').fit(df)

In [63]:
df = catEncoder.transform(df)

In [64]:
df.show(3)

+---+---+----+----+----+----+---+-----+----+-----+----+----+-------------+------+------+
|Age|Sex| ALB| ALP| ALT| AST|BIL|  CHE|CHOL| CREA| GGT|PROT|     Category|Gender|Target|
+---+---+----+----+----+----+---+-----+----+-----+----+----+-------------+------+------+
| 32|  m|38.5|52.5| 7.7|22.1|7.5| 6.93|3.23|106.0|12.1|  69|0=Blood Donor|   0.0|   0.0|
| 32|  m|38.5|70.3|  18|24.7|3.9|11.17| 4.8| 74.0|15.6|76.5|0=Blood Donor|   0.0|   0.0|
| 32|  m|46.9|74.7|36.2|52.6|6.1| 8.84| 5.2| 86.0|33.2|79.3|0=Blood Donor|   0.0|   0.0|
+---+---+----+----+----+----+---+-----+----+-----+----+----+-------------+------+------+
only showing top 3 rows



In [65]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- ALB: string (nullable = true)
 |-- ALP: string (nullable = true)
 |-- ALT: string (nullable = true)
 |-- AST: double (nullable = true)
 |-- BIL: double (nullable = true)
 |-- CHE: double (nullable = true)
 |-- CHOL: string (nullable = true)
 |-- CREA: double (nullable = true)
 |-- GGT: double (nullable = true)
 |-- PROT: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Gender: double (nullable = false)
 |-- Target: double (nullable = false)



In [66]:
catEncoder.labels

['0=Blood Donor',
 '3=Cirrhosis',
 '1=Hepatitis',
 '2=Fibrosis',
 '0s=suspect Blood Donor']

In [67]:
from pyspark.ml.feature import IndexToString

In [68]:
converter = IndexToString(inputCol='Target', outputCol='orig_cat')

In [69]:
## go back ..return original column
converted_df = converter.transform(df)

In [70]:
converted_df.show(4)

+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+------+------+-------------+
|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|     Category|Gender|Target|     orig_cat|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+------+------+-------------+
| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|  69|0=Blood Donor|   0.0|   0.0|0=Blood Donor|
| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|0=Blood Donor|   0.0|   0.0|0=Blood Donor|
| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|0=Blood Donor|   0.0|   0.0|0=Blood Donor|
| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|0=Blood Donor|   0.0|   0.0|0=Blood Donor|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+------+------+-------------+
only showing top 4 rows



**Feature** 
+ Selection numerical column and drop string column

In [72]:
print(df.columns)

['Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'Category', 'Gender', 'Target']


In [73]:
df2 = df.select('Age', 'Gender', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'Target')

In [74]:
df2 = df2.toPandas().replace('NA', 0).astype(float)

In [75]:
type(df2)

pandas.core.frame.DataFrame

In [76]:
#convert df2 (pandas to spark dataframe)
new_df = spark.createDataFrame(df2)

In [77]:
new_df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Gender: double (nullable = true)
 |-- ALB: double (nullable = true)
 |-- ALP: double (nullable = true)
 |-- ALT: double (nullable = true)
 |-- AST: double (nullable = true)
 |-- BIL: double (nullable = true)
 |-- CHE: double (nullable = true)
 |-- CHOL: double (nullable = true)
 |-- CREA: double (nullable = true)
 |-- GGT: double (nullable = true)
 |-- PROT: double (nullable = true)
 |-- Target: double (nullable = true)



In [78]:
## required columns

required_columns = ['Age', 'Gender', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'Target']

+ Vectorization columns

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees

In [79]:
vect_assembler = VectorAssembler(inputCols=required_columns, outputCol='features')

In [87]:
new_df = vect_assembler.transform(new_df)

In [88]:
new_df.show(5)

+----+------+----+----+----+----+----+-----+----+-----+----+----+------+--------------------+
| Age|Gender| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|Target|            features|
+----+------+----+----+----+----+----+-----+----+-----+----+----+------+--------------------+
|32.0|   0.0|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|69.0|   0.0|[32.0,0.0,38.5,52...|
|32.0|   0.0|38.5|70.3|18.0|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|   0.0|[32.0,0.0,38.5,70...|
|32.0|   0.0|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|   0.0|[32.0,0.0,46.9,74...|
|32.0|   0.0|43.2|52.0|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|   0.0|[32.0,0.0,43.2,52...|
|32.0|   0.0|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|   0.0|[32.0,0.0,39.2,74...|
+----+------+----+----+----+----+----+-----+----+-----+----+----+------+--------------------+
only showing top 5 rows



### Train and Test Split

In [89]:
train_df, test_df = new_df.randomSplit([0.7, 0.3],seed=23)

### Model building

In [90]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

In [91]:
### logistic model

lr = LogisticRegression(featuresCol='features', labelCol='Target')

### Train model

In [92]:
lr_model = lr.fit(train_df)

### Prediction on test dataframe

In [93]:
## Prediction

y_pred = lr_model.transform(test_df)
y_pred.show(10)

+----+------+----+----+----+----+----+-----+----+-----+----+----+------+--------------------+--------------------+--------------------+----------+
| Age|Gender| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|Target|            features|       rawPrediction|         probability|prediction|
+----+------+----+----+----+----+----+-----+----+-----+----+----+------+--------------------+--------------------+--------------------+----------+
|32.0|   0.0|42.2|41.9|35.8|31.1|16.1| 5.82| 4.6|109.0|21.5|67.1|   0.0|[32.0,0.0,42.2,41...|[391.169028224589...|[1.0,6.6809237325...|       0.0|
|33.0|   0.0|36.3|78.6|23.6|22.0| 7.0| 8.56|5.38| 78.0|19.4|68.7|   0.0|[33.0,0.0,36.3,78...|[501.959256810496...|[1.0,1.1031498125...|       0.0|
|33.0|   0.0|36.6|57.1|38.9|40.3|24.9| 9.62| 5.5|112.0|27.6|69.3|   0.0|[33.0,0.0,36.6,57...|[402.124713027645...|[1.0,4.9572075156...|       0.0|
|33.0|   0.0|38.7|39.8|22.5|23.0| 4.1| 4.63|4.97| 63.0|15.2|71.9|   0.0|[33.0,0.0,38.7,39...|[411.884692528756...|[1.0

In [95]:
print(y_pred.columns)

['Age', 'Gender', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'Target', 'features', 'rawPrediction', 'probability', 'prediction']


In [96]:
y_pred.select('Target','rawPrediction', 'probability', 'prediction').show(10)

+------+--------------------+--------------------+----------+
|Target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[391.169028224589...|[1.0,6.6809237325...|       0.0|
|   0.0|[501.959256810496...|[1.0,1.1031498125...|       0.0|
|   0.0|[402.124713027645...|[1.0,4.9572075156...|       0.0|
|   0.0|[411.884692528756...|[1.0,2.6310159611...|       0.0|
|   0.0|[526.735040008043...|[1.0,5.2563173708...|       0.0|
|   0.0|[453.389430737212...|[1.0,8.7275314142...|       0.0|
|   0.0|[493.726503831349...|[1.0,9.5701594837...|       0.0|
|   0.0|[530.784079956260...|[1.0,1.4705777321...|       0.0|
|   0.0|[543.287202308801...|[1.0,2.6136571724...|       0.0|
|   0.0|[410.457035325134...|[1.0,2.1332325291...|       0.0|
+------+--------------------+--------------------+----------+
only showing top 10 rows



### Evaluate my model

In [97]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol='Target', metricName='accuracy')

In [98]:
multi_evaluator.evaluate(y_pred)

0.9774011299435028

In [99]:
from pyspark.mllib.evaluation import MulticlassMetrics

lr_metrics = MulticlassMetrics(y_pred['Target','prediction'].rdd)

In [100]:
print("Accuracy", lr_metrics.accuracy)

Accuracy 0.9774011299435028


## Save Model

In [133]:
#! mkdir models

In [102]:
import pickle

In [ ]:
filename = 'lr_model.sav'
pickle.dump(lr_model, open("models/"+filename, 'wb'))